In [3]:
import pymysql.cursors
#import jsonify
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.ensemble import RandomForestRegressor

In [5]:
conn = pymysql.connect(
    host="softwaredb.ce0otalnccc9.eu-west-1.rds.amazonaws.com",user="soft",password="password",database="dublinbikes",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)


In [6]:
availability_query = "SELECT number, time, available_bikes, available_bike_stands FROM availability"
#weather_query = "SELECT number, time, temperature, wind_speed, wind_direction, weather_code FROM weather"
weather_query = "SELECT * FROM weather"
# Load the query results into pandas dataframes
availability_df = pd.read_sql_query(availability_query, conn)
weather_df = pd.read_sql_query(weather_query, conn)

# Merge the dataframes on "time" and "number" columns



# Close the database connection
conn.close()

C:\Users\tfiro\AppData\Local\Temp\ipykernel_4836\2178617842.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  availability_df = pd.read_sql_query(availability_query, conn)
C:\Users\tfiro\AppData\Local\Temp\ipykernel_4836\2178617842.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weather_df = pd.read_sql_query(weather_query, conn)


In [7]:
availability_df['time'] = availability_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
weather_df['time'] = weather_df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
merged_df = pd.merge(availability_df, weather_df, on=["number", "time"])

In [8]:
merged_df.to_csv('merged_data.csv', index=False)

In [40]:
merged_df.tail()

,number,time,available_bikes,available_bike_stands,temperature,wind_speed,wind_direction,weather_code,precipitation_sum,rain_sum,precipitation_probability,hour,minute,month,day
1145860,507,2023-03-10 16:40:02,1,0,6,3,76,1,8.1,3.1,100,16,40,3,10
1145861,507,2023-03-10 16:45:03,1,0,6,3,76,1,8.1,3.1,100,16,45,3,10
1145862,507,2023-03-10 16:50:03,1,0,6,3,76,1,8.1,3.1,100,16,50,3,10
1145863,507,2023-03-10 16:55:03,1,0,6,3,76,1,8.1,3.1,100,16,55,3,10
1145864,507,2023-03-10 17:00:07,1,0,5,8,90,1,8.1,3.1,100,17,0,3,10


In [27]:
merged_df.isnull().sum()

number                       0
time                         0
available_bikes              0
available_bike_stands        0
temperature                  0
wind_speed                   0
wind_direction               0
weather_code                 0
precipitation_sum            0
rain_sum                     0
precipitation_probability    0
dtype: int64

In [27]:
merged_df.describe().T

,count,mean,std,min,25%,50%,75%,max
available_bikes,1145865.0,11.567685,9.406132,0.0,4.0,10.0,17.0,40.0
available_bike_stands,1145865.0,20.024614,10.882782,0.0,12.0,20.0,29.0,40.0
temperature,1145865.0,9.044022,3.237408,1.0,7.0,10.0,11.0,17.0
wind_speed,1145865.0,17.026081,8.261317,1.0,11.0,16.0,22.0,44.0
wind_direction,1145865.0,196.005774,74.370822,3.0,145.0,208.0,254.0,360.0
precipitation_sum,1145865.0,3.587186,5.234813,0.0,0.3,1.9,3.7,30.2
rain_sum,1145865.0,2.608202,3.958213,0.0,0.1,0.7,3.3,20.2
precipitation_probability,1145865.0,74.635592,33.036792,0.0,58.0,90.0,100.0,100.0


In [28]:
print(merged_df.dtypes)


number                             category
time                         datetime64[ns]
available_bikes                       int64
available_bike_stands                 int64
temperature                           int64
wind_speed                            int64
wind_direction                        int64
weather_code                       category
precipitation_sum                   float64
rain_sum                            float64
precipitation_probability             int64
hour                               category
minute                             category
month                              category
day                                category
dtype: object


In [22]:
merged_df['time'] = merged_df['time'].astype('datetime64')
merged_df['number'] = merged_df['number'].astype('category')
merged_df['weather_code'] = merged_df['weather_code'].astype('category')

C:\Users\tfiro\AppData\Local\Temp\ipykernel_4836\2672415241.py:1: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  merged_df['time'] = merged_df['time'].astype('datetime64')


In [39]:
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute
merged_df['month'] = merged_df['time'].dt.month
merged_df['day'] = merged_df['time'].dt.day


In [24]:
merged_df['hour'] = merged_df['time'].astype('category')
merged_df['minute'] = merged_df['time'].astype('category')
merged_df['month'] = merged_df['time'].astype('category')
merged_df['day'] = merged_df['time'].astype('category')

In [41]:
station_dfs = {}
for station in merged_df['number'].unique():
    station_dfs[station] = merged_df[merged_df['number'] == station]


In [47]:
filtered_df = merged_df[merged_df['number'] == 88]
filtered_df

,number,time,available_bikes,available_bike_stands,temperature,wind_speed,wind_direction,weather_code,precipitation_sum,rain_sum,precipitation_probability,hour,minute,month,day


In [38]:
from sklearn.ensemble import RandomForestRegressor
for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    station_num = 1
    time_str = '2023-03-07 19:35:03'
    weather_code = 801
    temperature = 10
    wind_speed = 6
    wind_direction = 240
    time = pd.to_datetime(time_str)
    hour = time.hour
    day=time.day
    minute=time.minute
    month=time.month
    prediction_input = [[hour,day,minute,month, temperature, wind_speed, wind_direction, weather_code]]
    prediction = model.predict(prediction_input)
    
    print(f"Predicted number of available bikes for Station {station} at {time_str}: {prediction[0]:.2f}")


TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [43]:


for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bikes']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    
    #Save the model to a pickle file
    filename = f"model_dep{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)


Station 1 R-squared: 0.9854539615991851
Station 2 R-squared: 0.967574552269479
Station 3 R-squared: 0.9818897856084019
Station 4 R-squared: 0.9885978611411956
Station 5 R-squared: 0.9860923068817139
Station 6 R-squared: 0.9890663416192932
Station 7 R-squared: 0.9899008798317374
Station 8 R-squared: 0.9812109285394759
Station 9 R-squared: 0.9734360852872317
Station 10 R-squared: 0.9668055771390129
Station 11 R-squared: 0.995013876407448
Station 12 R-squared: 0.984494343358818
Station 13 R-squared: 0.9938812717733768
Station 14 R-squared: 0.9786129134366893
Station 15 R-squared: 0.958672883704536
Station 16 R-squared: 0.9840213836858838
Station 17 R-squared: 0.9901358763476128
Station 18 R-squared: 0.9863112884766161
Station 19 R-squared: 0.9890896264029799
Station 20 R-squared: 0.9933336998064097
Station 21 R-squared: 0.9879062331214817
Station 22 R-squared: 0.9860261504615114
Station 23 R-squared: 0.9822690387836451
Station 24 R-squared: 0.9663712161835871
Station 25 R-squared: 0.99369

In [19]:
import pickle
from sklearn.ensemble import RandomForestRegressor

for station, station_df in station_dfs.items():
    X = station_df[['hour', 'day','minute','month','temperature', 'wind_speed', 'wind_direction', 'weather_code']]
    y = station_df['available_bike_stands']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    print(f"Station {station} R-squared: {model.score(X_test, y_test)}")
    
    #Save the model to a pickle file
    filename = f"model_arr{station}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(model, file, pickle.HIGHEST_PROTOCOL)


TypeError: float() argument must be a string or a real number, not 'Timestamp'